In [ ]:
%matplotlib inline
from matplotlib.pyplot import style


style.use("seaborn-notebook")

# Accessing IOOS CO-OPS.NOS data via ERDDAP

<https://tidesandcurrents.noaa.gov/about.html>

In [ ]:
from erddapy import ERDDAP

server = "https://opendap.co-ops.nos.noaa.gov/erddap"
e = ERDDAP(server=server, protocol="tabledap")

In [ ]:
dataset_id = "IOOS_Hourly_Height_Verified_Water_Level"

e.dataset_id = dataset_id

The `dataset_id` can be found by searching the server with the `get_search_url` or with the browser.

We can check what is in the dataset with ERDDAP's `info` URL.

In [ ]:
import pandas as pd

info_url = e.get_info_url(dataset_id=dataset_id, response="csv")

info = pd.read_csv(info_url)
info.head()

The variables are:

In [ ]:
set(info["Variable Name"])

This server have some `REQUIRED` variables for each request to get a valid URL,

In [ ]:
info.loc[info["Value"].astype(str).str.startswith("REQUIRED")]

and the time has its own special format.

In [ ]:
pd.set_option("max_colwidth", 100)

info.loc[info["Variable Name"] == "END_DATE"]["Value"].iloc[-1]

[With most ERDDAP servers](https://ioos.github.io/erddapy/) we can pass the time constraint as any valid time string or a python object, this dataset uses `yyyymmdd` for some reason :-/

Also, because this server has some **required** variables, and the time constraints does not work here, we must pass the actual variable values as constraints.

In [ ]:
e.constraints = {
    "BEGIN_DATE=": "20140801",
    "END_DATE=": "20140901",
    "DATUM=": "MLLW",
    "STATION_ID=": "8570283",
}

url = e.get_download_url()

print(url)

We can request the data from that URL as netCDF, CSV, JSON, etc. B/c it is a time-series let's request a `csv` and put it in a pandas `DataFrame` via the `to_pandas` method.

In [ ]:
df = e.to_pandas(index_col="time (UTC)", parse_dates=True,).dropna()

# Make it a naive time zone for plotting.
df.index = df.index.tz_localize(None)

df.head()

In [ ]:
df["WL_VALUE (meters)"].plot(figsize=(15, 3.75));

Exercise:

- Change the constraints below and create a similar plot. Start with the dates.
- Try to find another stations (STATION_ID) by browsing the [dataset online](https://opendap.co-ops.nos.noaa.gov/erddap). Was that easy?

```python
e.constraints = {
    "BEGIN_DATE=": "20140801",
    "END_DATE=": "20140901",
    "DATUM=": "MLLW",
    "STATION_ID=": "8570283",
}

url = e.get_download_url()

```